In [3]:
import numpy as np
import matplotlib.pyplot as plt
import random
import uuid

In [64]:
class Node:
    def __init__(self):
        self.id = uuid.uuid1()


class InputNode(Node):
    def __init__(self, var_id):
        super().__init__()
        self.var_id = var_id
        self.children = []


class AndNode(Node):
    def __init__(self, var_id):
        super().__init__()
        self.var_id = var_id
        self.is_output = False
        self.parents = []
        self.children = []


class NodeNetwork:
    def __init__(self, num_inputs, hidden_layers, num_outputs):
        self.num_inputs = num_inputs
        self.hidden_layers = hidden_layers
        self.num_outputs = num_outputs
        self.max_var_idx = 0
        self.nodes_ = []

        for i in range(len(self.hidden_layers) + 1):
            if i == 0:
                input_nodes = []
                for i in range(num_inputs):
                    self.max_var_idx += 1
                    inp_node = InputNode(self.max_var_idx)
                    inp_node.loc = (0, i)
                    input_nodes.append(inp_node)
                self.nodes_.append(input_nodes)
            else:
                and_nodes = []
                for j in range(self.hidden_layers[i - 1]):
                    self.max_var_idx += 1
                    and_node = AndNode(self.max_var_idx)
                    and_node.loc = (i, j)
                    k = 0
                    while len(and_node.parents) < 2:
                        candidate_parent = self.nodes_[-1][k % len(self.nodes_[-1])]
                        if (
                            (
                                len(candidate_parent.children)
                                == 0
                                or random.choice([True, False])
                            )
                            and candidate_parent.loc
                            not in and_node.parents
                        ):
                            and_node.parents.append(
                                candidate_parent.loc
                            )
                            candidate_parent.children.append(
                                and_node.loc
                            )
                        k += 1
                    and_nodes.append(and_node)
                self.nodes_.append(and_nodes)
        # Currently only last layer nodes are outputs
        for node in self.nodes_[-1]:
            node.is_output = True

In [66]:
nn = NodeNetwork(4, [3, 3, 2], 2)

out_str = (
    f"aag {nn.max_var_idx} {nn.num_inputs} 0 {nn.num_outputs} {sum(nn.hidden_layers)}\n"
)

for node in nn.nodes_[0]:
    out_str += f"{node.var_id * 2}\n"

for node in nn.nodes_[-1]:
    out_str += f"{node.var_id * 2}\n"

for i in range(len(nn.nodes_)):
    if i == 0:
        continue
    for upper_node in nn.nodes_[i]:
        idx00 = upper_node.parents[0][0]
        idx01 = upper_node.parents[0][1]
        idx10 = upper_node.parents[1][0]
        idx11 = upper_node.parents[1][1]
        out_str += f"{2 * upper_node.var_id} {nn.nodes_[idx00][idx01].var_id * 2} {nn.nodes_[idx10][idx11].var_id * 2}\n"

print(out_str)

aag 12 4 0 2 8
2
4
6
8
22
24
10 2 4
12 4 6
14 8 2
16 10 12
18 12 14
20 10 14
22 16 18
24 16 20



In [68]:
!aiger/aigvis aiger/my_examples/gen.aag